In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
from google.colab import drive

In [ ]:
# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# The collection of functions used in this project

In [ ]:
def events_to_image(xs, ys, ps, sensor_size=(260, 346)):
    """
    Place events into an image using numpy
    @param xs x coords of events
    @param ys y coords of events
    @param ps Event polarities/weights
    @param sensor_size The size of the event camera sensor
    @returns Event image from the input events
    """

    event_frame = np.zeros(shape=(sensor_size[0], sensor_size[1]), dtype=np.int8)
    event_frame[ys, xs] = np.where(ps == 1, 1, -1)

    return event_frame

In [ ]:
def events_to_voxel(xs, ys, ts, ps, B, sensor_size=(260, 346)):
    """
    Turn set of events to a voxel grid tensor, using temporal bilinear interpolation
    @param xs List of event x coordinates (torch tensor)
    @param ys List of event y coordinates (torch tensor)
    @param ts List of event timestamps (torch tensor)
    @param ps List of event polarities (torch tensor)
    @param B Number of bins in output voxel grids (int)
    @param sensor_size The size of the event sensor/output voxels
    @returns Voxel of the events between t0 and t1
    """
    voxel_grid = np.zeros((B, sensor_size[0], sensor_size[1]), np.float32).ravel()
    
    # normalize the event timestamps so that they lie between 0 and num_bins
    deltaT = np.absolute(ts[-1] - ts[0])
    
    if deltaT == 0:
        deltaT = 1.0
    
    ts = (B - 1) * (ts - ts[0]) / deltaT
    
    tis = ts.astype(int)
    dts = ts - tis
    vals_left = ps * (1.0 - dts)
    vals_right = ps * dts
    
    valid_indices = tis < B
    np.add.at(voxel_grid, xs[valid_indices] + ys[valid_indices] * sensor_size[1] +
              tis[valid_indices] * sensor_size[1] * sensor_size[0], vals_left[valid_indices])
    
    valid_indices = (tis + 1) < B
    np.add.at(voxel_grid, xs[valid_indices] + ys[valid_indices] * sensor_size[1] +
              (tis[valid_indices] + 1) * sensor_size[1] * sensor_size[0], vals_right[valid_indices])
    
    voxel_grid = np.reshape(voxel_grid, (B, sensor_size[0], sensor_size[1]))
    
    return voxel_grid

# The creation of the voxel grid tensors

In [ ]:
path = os.getcwd() + '/drive/My Drive/My Final Year Project/Training data/'

In [ ]:
# Load the x-coords of events
#path_to_events_xs = path + 'events_xs.npy'
#events_xs = np.load(file=path_to_events_xs)

#Load the y-coords of events
#path_to_events_ys = path + 'events_ys.npy'
#events_ys = np.load(file=path_to_events_ys)

#Load the timestamps of events
path_to_events_ts = path + 'events_ts.npy'
events_ts = np.load(file=path_to_events_ts)

#Load the polarity p of events
#path_to_events_ps = path + 'events_ps.npy'
#events_ps = np.load(file=path_to_events_ps)

In [ ]:
hold_steps = np.arange(0, 400000000, 1000000, dtype=np.uint32)
hold_voxel_grids = np.empty(shape=(400, 3, 260, 346), dtype=np.float32)

In [ ]:
for i in range(0, 399):
    hold_voxel_grids[i] = events_to_voxel(xs=events_xs[hold_steps[i]:hold_steps[i+1]], ys=events_ys[hold_steps[i]:hold_steps[i+1]], ts=events_ts[hold_steps[i]:hold_steps[i+1]], ps=events_ps[hold_steps[i]:hold_steps[i+1]], B=3)

In [ ]:
hold_voxel_grids = np.moveaxis(a=hold_voxel_grids, source=1, destination=-1)

In [ ]:
# save voxel_grids as npy files
np.save(file=os.getcwd() + '/drive/My Drive/My Final Year Project/' + 'voxel_grids.npy', arr=hold_voxel_grids)

In [ ]:
# Load the ododmetry
path_to_odometry = path + 'odometry.npy'
odometry = np.load(file=path_to_odometry)

# Load the odometry timestamps
path_to_odometry_ts = path + 'odometry_ts.npy'
odometry_ts = np.load(file=path_to_odometry_ts)

In [ ]:
odometry_ts = odometry_ts - odometry_ts[0]

In [ ]:
odometry_ts = odometry_ts.astype(np.float32)

In [ ]:
odometry_ts = np.around(a=odometry_ts, decimals=1)

In [ ]:
for ts in odometry_ts:
    print(ts)

In [ ]:
# Load image timestamps image_ts
path_to_image_ts = path + 'image_ts.npy'
image_ts = np.load(file=path_to_image_ts)

In [ ]:
# Load odometry timestamps odometry_ts
path_to_odometry_ts = path + 'odometry_ts.npy'
odometry_ts = np.load(file=path_to_odometry_ts)

# Load odometry data 
path_to_odometry = path + 'odometry.npy'
odometry = np.load(file=path_to_odometry)

In [ ]:
# Load pose timestamps pose_ts
path_to_pose_ts = path + 'pose_ts.npy'
pose_ts = np.load(file=path_to_pose_ts)

# Load pose data 
path_to_pose = path + 'pose.npy'
pose = np.load(file=path_to_pose)

In [ ]:
hold_image_ts = []
for i in range(0, 399):
    hold_image_ts.append([events_ts[hold_steps[i]:hold_steps[i+1]][0],events_ts[hold_steps[i]:hold_steps[i+1]][-1]])

In [ ]:
hold_image_ts = np.asarray(hold_image_ts)

In [ ]:
hold_image_ts.shape

(399, 2)

In [ ]:
for i, im_ts in enumerate(hold_image_ts):
    print(i, ":", im_ts)

In [ ]:
u_ts, u_ind = np.unique(image_ts, return_index=True)

In [ ]:
for ind in u_ind:
    print(ind, ':', u_ts[ind])

In [ ]:
for i, ts in enumerate(odometry_ts):
    if i > 100 or i < 1000:
        print(i,":", np.around(ts - odometry_ts[0], 1))
    if i == 1001:
        break

In [ ]:
for i, ts in enumerate(pose_ts):
    if i > 100 or i < 1000:
        print(i,":", np.around(ts - pose_ts[0], 1))
    if i == 1001:
        break

In [ ]:
#np.save(file=os.getcwd() + '/drive/My Drive/My Final Year Project/' + 'voxel_grids.npy', arr=hold_voxel_grids)

In [ ]:
#np.save(file=os.getcwd() + '/drive/My Drive/My Final Year Project/' + 'events_ts.npy', arr=new_event_ts)